# Initialize the code

In [31]:

install.packages("glmnet")

also installing the dependencies 'lattice', 'codetools', 'iterators', 'Matrix', 'foreach', 'shape'




  There is a binary version available but the source version is later:
       binary source needs_compilation
Matrix 1.2-17 1.2-18              TRUE

  Binaries will be installed
package 'lattice' successfully unpacked and MD5 sums checked
package 'codetools' successfully unpacked and MD5 sums checked
package 'iterators' successfully unpacked and MD5 sums checked
package 'Matrix' successfully unpacked and MD5 sums checked
package 'foreach' successfully unpacked and MD5 sums checked
package 'shape' successfully unpacked and MD5 sums checked
package 'glmnet' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\tirya\AppData\Local\Temp\RtmpC0h2D1\downloaded_packages


In [29]:
library(readxl)

In [28]:
library(readxl)
library(glmnet)
MAPE = function(actual, predicted){
  result = (sum(abs(actual - predicted)/actual)*100)/length(actual)
  return(result)
}

ERROR: Error in library(glmnet): there is no package called 'glmnet'


## Results

In [30]:
setwd("C:/Users/tirya/Desktop/IE 582/Homework 3")
mydata <- read_excel('RealTimeConsumption-01012016-17112019.xlsx', sheet=1) # I convert all the names to English
mydata = data.frame(mydata)
str(mydata)

'data.frame':	33984 obs. of  7 variables:
 $ Date       : POSIXct, format: "2016-01-01" "2016-01-01" ...
 $ Hour_Real  : POSIXct, format: "1899-12-31 00:00:00" "1899-12-31 01:00:00" ...
 $ Hour       : num  0 1 2 3 4 5 6 7 8 9 ...
 $ Day        : chr  "Friday" "Friday" "Friday" "Friday" ...
 $ Lag_48     : num  NA NA NA NA NA NA NA NA NA NA ...
 $ Lag_168    : num  NA NA NA NA NA NA NA NA NA NA ...
 $ Consumption: num  26277 24992 23533 22465 22003 ...
